In [14]:
# This code is to calculate kdPT (y) and PT (z) for C. crescentus based on eq (s44) and (s45) in the supplementary text.

import numpy as np
import pandas as pd

# x grid
x_std, x_mean = 0.001554, 0.011753 #experimentally measured kd1* mean and std in sec-1
grid = np.random.normal(x_mean, x_std, size=1000)

# constants, set up std and mean here for a,b,t,l
a_std, a_mean = 0.08, 0.16     # (N5-N3)/N5 measured from steady-state Z5 and Z3 levels +/- 0.08 
b_std, b_mean = 0.00217, 0.021 # kd1, measured from rif experiment (-BCM). 1.28/60 +/-0.13/60 (in sec-1)
t_std, t_mean = 8, 69          # tau or transcription time/2 (in sec) transcription time = 2.3 ± 0.27 min
l_std, l_mean = 7, 158         # 1/kd2 of Z3 (in sec)  kd2= 0.38 +/ 0.0374 min-1 

# number of simulations
N = 1000

def solution(a, b, t, l, iteration_num):
    def u(x):
        return np.exp(-x * t)
   
    A = a - 1
    L = lambda x: A * (1 - u(x)**2) + l * x
    M = lambda x: A * u(x) * (1-u(x)) + l * x
    N = lambda x: A * x * u(x)
   
    K = lambda x: (M(x) * x - N(x) * u(x)) / L(x) / u(x)
    J = lambda x: - M(x) * (x - b) / L(x) / u(x)
   
    P = lambda x: u(x) * K(x)
    Q = lambda x: u(x) * J(x) - x
    R = lambda x: x - b
   
    D = lambda x: Q(x)**2 - 4 * P(x) * R(x)
   
    z1 = lambda x: (-Q(x) + np.sqrt(D(x))) / 2 / P(x)
    z2 = lambda x: (-Q(x) - np.sqrt(D(x))) / 2 / P(x)
   
    y1 = lambda x: K(x) * z1(x) + J(x)
    y2 = lambda x: K(x) * z2(x) + J(x)
   
   
    df = pd.DataFrame([grid, [y1(x) for x in grid], [z1(x) for x in grid], [y2(x) for x in grid], [z2(x) for x in grid]], index=['x', 'y1', 'z1', 'y2', 'z2']).T
    df.loc[:, 'experiment'] = iteration_num
    return df

def sample():
    return np.random.normal(a_mean, a_std), np.random.normal(b_mean, b_std), np.random.normal(t_mean, t_std), np.random.normal(l_mean, l_std)

# run simulation
np.random.seed(1234)
constants = [sample() for _ in range(N)]

res = []
for i, c in enumerate(constants):
    a, b, t, l = c
    res.append(solution(a, b, t, l, i))
res_df = pd.concat(res)

# statistics
# we get two (y,z) pair for each x, but one pair is negative and not biologically relevant.
res_df[['y1', 'z1', 'y2', 'z2']].mean()


y1    0.057426
z1    0.685373
y2   -0.001357
z2   -0.757062
dtype: float64

In [15]:
res_df[['y1', 'z1', 'y2', 'z2']].std()

y1    0.010176
z1    0.044162
y2    0.000476
z2    0.277543
dtype: float64